## Librerias

In [3]:
import re
import spacy
import random
from spacy.training import Example
from spacy.util import minibatch, compounding
from typing import Optional
import logging
import os
import json
import unicodedata

In [4]:
# Configura el logging al principio del script (o en un archivo de configuración separado)
logging.basicConfig(level=logging.INFO,  # Nivel de detalle (INFO, DEBUG, WARNING, ERROR, CRITICAL)
                    format='%(asctime)s - %(levelname)s - %(message)s')

## Funciones de limpieza OCR

In [5]:
def fix_ocr_issues_for_training(text: str) -> str:
    """
    Limpia el texto para entrenamiento, con manejo de Unicode y regex más precisas.
    """
    # Normalizar caracteres Unicode
    text = unicodedata.normalize("NFKC", text)

    # Regex para fechas (más robusta)
    text = re.sub(r"(?P<dia>\d{1,2})\s*(?P<mes>ENE|FEB|MAR|ABR|MAY|JUN|JUL|AGO|SEP|OCT|NOV|DIC)\s*(?P<año>\d{4})",
                r"\g<dia>\g<mes>\g<año>", text, flags=re.IGNORECASE)
    # Unir digitos separados en las fechas
    text = re.sub(r"(?P<dia>\d{1,2})\s+(?P<mes>ENE|FEB|MAR|ABR|MAY|JUN|JUL|AGO|SEP|OCT|NOV|DIC)\s+(?P<año>\d{4})",
            r"\g<dia>\g<mes>\g<año>", text, flags=re.IGNORECASE)

    # Quitar saltos de línea extraños dentro de palabras
    text = re.sub(r"(\w+)\s+-\s+(\w+)", r"\1\2", text) #e.g., "mani-  festación" -> "manifestación"
    text = re.sub(r"(\w+)\n(\w+)", r"\1\2", text) #e.g., "mani\nfes" -> "manifes"
    text = re.sub(r"(\w+)-(\n)(\w+)", r"\1\3", text) #para problema de guion
    #Eliminar espacios extras
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"[:;,.]+", r". ", text)

    return text

## Context

In [6]:
raw_text_list = [
    """2353 17ABR2024 LEY No. "POR MEDIO DE LA CUAL...""",
    """2355 2024 LEY No. 17MAY "POR MEDIO DE LA CUAL...""",
    # ...
]

def add_new_examples(raw_text_list, train_data, output_file="train_data.json", interactive=True):
    """
    Agrega nuevos ejemplos, los anota (manualmente o no) y guarda TRAIN_DATA.

    Args:
        raw_text_list: Lista de textos crudos.
        train_data: Lista de entrenamiento actual (se modifica in-place).
        output_file: Nombre del archivo JSON para guardar.
        interactive:  Si es True, pide anotación manual. Si es False,
                      asume que las entidades ya están en raw_text_list.
    """
    for raw_text in raw_text_list:
        cleaned_text = fix_ocr_issues_for_training(raw_text)

        if interactive:
            print("=== Texto limpio para anotar ===")
            print(cleaned_text)
            print("Longitud:", len(cleaned_text))

            entities = []
            while True:
                try:
                    start = int(input("Inicio de entidad (o -1 para terminar): "))
                    if start == -1:
                        break
                    end = int(input("Fin de entidad: "))
                    label = input("Etiqueta: ")
                    entities.append((start, end, label))
                except ValueError:
                    print("Entrada inválida.  Ingresa números enteros.")
        else:
            # Modo no interactivo.  Busca si ya existe y usa sus entidades.
            found = False
            for i, (existing_text, annotations) in enumerate(train_data):
                if existing_text == cleaned_text:
                    entities = annotations["entities"]
                    found = True
                    del train_data[i]  # Lo borra (por si hay duplicados)
                    break
            if not found:
                print(f"WARNING: Texto '{cleaned_text}' no encontrado. Se añade sin entidades.")
                entities = []

        train_data.append((cleaned_text, {"entities": entities}))

    # Guardar TRAIN_DATA en un archivo JSON
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(train_data, f, ensure_ascii=False, indent=4)
        print(f"TRAIN_DATA actualizado guardado en {output_file}")
    except Exception as e:
        logging.error(f"Error al guardar TRAIN_DATA: {e}")

    return train_data

### Para cargar TRAIN_DATA desde el archivo:

In [7]:
def load_train_data(input_file="train_data.json"):
    """Carga los datos de entrenamiento desde un archivo JSON."""
    try:
        with open(input_file, "r", encoding="utf-8") as f:
            train_data = json.load(f)
            return train_data
    except FileNotFoundError:
        logging.warning(f"Archivo de datos de entrenamiento no encontrado: {input_file}.  Se usará una lista vacía.")
        return []
    except json.JSONDecodeError:
        logging.error(f"Error al decodificar el archivo JSON: {input_file}.  Verifica el formato.")
        return []
    except Exception as e:
        logging.error(f"Error inesperado al cargar TRAIN_DATA: {e}")
        return []

## load the data

In [ ]:
# 1. Carga los datos existentes (si los hay)
TRAIN_DATA = load_train_data()
if not TRAIN_DATA:
    TRAIN_DATA = [
        (
            '2182 3ENE2022 LEY No "POR MEDIO DEL CUAL SE MODIFICA EL ARTICULO 13 DE LA LEY ) 2002" EL CONGRESO DE COLOMBIA',
            {"entities": [
                (0, 4, "NUMERO_LEY"),   # "2182"
                (5, 13, "FECHA"),      # "3ENE2022"
                (14, 103, "EPIGRAFE") # "LEY No  POR MEDIO DEL CUAL SE MODIFICA EL ARTICULO 13 DE LA LEY ) 2002"
            ]}
        ),
        (
            '2184 6ENE2022 LEY No. POR MEDIO DE LA CUAL SE DICTAN NORMAS ENCAMINADAS A FOMENTAR, PROMOVER LA SOSTENIBILIDAD, LA VALORACION Y LA TRANSMISION DE LOS SABERES DE LOS OFICIOS ARTISTICOS, DE LAS INDUSTRIAS CREATIVAS Y CULTURALES, ARTESANALES Y DEL PATRIMONIO CULTURAL EN COLOMBIA Y SE DICTAN OTRAS DISPOSICIONES EL CONGRESO DE COLOMBIA',
            {"entities": [
                (0, 4, "NUMERO_LEY"),   # "2184"
                (5, 13, "FECHA"),      # "6ENE2022"
                (14, 236, "EPIGRAFE") # "LEY  No. POR  MEDIO  DE  LA  CUAL  SE  DICTAN  NORMAS  ENCAMINADAS  A  FOMENTAR,  PROMOVER  LA  SOSTENIBILIDAD,  LA  VALORACION  Y  LA  TRANSMISION  DE  LOS  SABERES  DE  LOS  OFICIOS  ARTISTICOS,  DE  LAS  INDUSTRIAS  CREATIVAS  Y  CULTURALES,  ARTESANALES  Y  DEL  PATRIMONIO  CULTURAL  EN  COLOMBIA  Y  SE  DICTAN  OTRAS  DISPOSICIONES"
            ]}
        ),
        (
            '2185 ENE2022 LEY No "POR MEDIO DE LA CUAL SE CREA EL FESTIVAL NACIONAL DE LA MARIMBA DE CHONTA, Y SE DICTAN OTRAS DISPOSICIONES" EL CONGRESO DE COLOMBIA',
            {"entities": [
                (0, 4, "NUMERO_LEY"),   # "2185"
                (5, 12, "FECHA"),      # "ENE2022" (Aquí *no* incluyes el día, o tendrias que modificar tu normalizacion)
                (13, 120, "EPIGRAFE")   # "LEY  No  POR  MEDIO  DE  LA  CUAL  SE  CREA  EL  FESTIVAL  NACIONAL DE  LA  MARIMBA  DE  CHONTA,  Y  SE  DICTAN  OTRAS  DISPOSICIONES"
            ]} 
        ),
    ]

# *nuevos* ejemplos *manualmente*, usa add_new_examples
#    en modo interactivo (sin el `interactive=False`):
nuevos_textos = [
   """8 FEB 2022
    - ...
    LEY ORGANICA No2199
    "POR MEDIO DE LA CUAL..." ...
    """,
]
TRAIN_DATA = add_new_examples(nuevos_textos, TRAIN_DATA) 

2025-03-19 12:37:09,167 - WARNING - Archivo de datos de entrenamiento no encontrado: train_data.json.  Se usará una lista vacía.


=== Texto limpio para anotar ===
8FEB2022 - .  LEY ORGANICA No2199 "POR MEDIO DE LA CUAL. " . 
Longitud: 61
Entrada inválida.  Ingresa números enteros.


## date Change

In [ ]:
def load_spacy_model(model_path="modelo_leyes_epigrafe"):
    return spacy.load(model_path)

def normalize_spacy_date(spacy_fecha: str) -> Optional[str]:
    """
    Normaliza fechas y registra errores.
    """
    month_map = {
        "ENE": "01", "FEB": "02", "MAR": "03", "ABR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AGO": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DIC": "12"
    }

    m = re.match(r"(\d{1,2})([A-Z]{3,4})(\d{4})", spacy_fecha.upper())
    if not m:
        logging.warning(f"No se pudo normalizar la fecha: {spacy_fecha}") # Registro
        return None  # O devolver la fecha original, según prefieras
    day = m.group(1).zfill(2)
    mes_str = m.group(2)
    year = m.group(3)
    mes_num = month_map.get(mes_str)
    if not mes_num:
        logging.warning(f"Mes desconocido en fecha: {spacy_fecha}") # Registro
        return None  # O devolver la fecha original
    return f"{day}/{mes_num}/{year}"


## train model

In [ ]:
def train_spacy_model(train_data, output_dir="modelo_leyes_epigrafe", n_iter=30, learning_rate=0.001, dropout_rate=0.2):

    # Crear un modelo en blanco para español e incorporar el componente de NER
    nlp = spacy.blank("es")
    ner = nlp.add_pipe("ner", last=True)

    # Añadir etiquetas al componente NER basándose en las anotaciones de train_data
    for _, annotations in train_data:
        for ent in annotations.get("entities", []):
            ner.add_label(ent[2])  # ent[2] corresponde a la etiqueta

    # Mezclar y dividir los datos en entrenamiento (80%) y validación (20%)
    random.shuffle(train_data)
    split = int(len(train_data) * 0.8)
    train_examples_raw = train_data[:split]
    dev_examples_raw = train_data[split:]


    # Preparar ejemplos de entrenamiento y validación
    def prepare_examples(data):
        examples = []
        for text, annotations in data:
            doc = nlp.make_doc(text)
            # Se espera que annotations tenga la estructura {"entities": [(inicio, fin, etiqueta), ...]}
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        return examples

    train_examples = prepare_examples(train_examples_raw)
    dev_examples = prepare_examples(dev_examples_raw)

    # Inicializar el entrenamiento; si se requiere ajustar la tasa de aprendizaje, 
    # se podría configurar el optimizador en base a "learning_rate" (actualmente no se utiliza)
    optimizer = nlp.begin_training()

    # Ciclo de entrenamiento
    for i in range(n_iter):
        random.shuffle(train_examples)
        losses = {}
        # Usar batches de tamaño dinámico
        batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            # Actualiza el modelo usando el batch actual, aplicando dropout para evitar overfitting
            nlp.update(batch, drop=dropout_rate, losses=losses, sgd=optimizer)
        print(f"Época {i + 1}, pérdidas: {losses}")

        # Evaluar el modelo en el conjunto de validación y mostrar resultados
        scores = nlp.evaluate(dev_examples)
        print(f"  Resultados en validación: {scores}")

    # Guardar el modelo entrenado en disco
    nlp.to_disk(output_dir)
    print(f"Modelo guardado en: {output_dir}")

    return nlp

# Entrenar
#nlp_trained = train_spacy_model(TRAIN_DATA, "modelo_leyes_epigrafe")


## ejemplo de uso

In [ ]:
# nlp_trained = load_spacy_model("modelo_leyes_epigrafe")

# test_text = "2353 17ABR2024 LEY No. ... El Congreso de Colombia,"
# test_text = fix_ocr_issues_for_training(test_text)  # Limpieza
# doc = nlp_trained(test_text)
# for ent in doc.ents:
#     if ent.label_ == "FECHA":
#         fecha_normalizada = normalize_spacy_date(ent.text)
#         print("Fecha final:", fecha_normalizada)
#     else:
#         print(ent.text, ent.label_)

2353 NUMERO_LEY
Fecha final: 17/04/2024


## Section final

In [ ]:
def spacy_extract_metadata(text: str, nlp) -> dict:
    # Limpiar el texto antes de procesarlo (se utiliza la función de corrección de OCR)
    text_clean = fix_ocr_issues_for_training(text)
    doc = nlp(text_clean)
    result = {
        "numero_ley": None,
        "fecha": None,
        "epigrafe": None
    }
    # Recorrer las entidades detectadas y asignarlas al diccionario de resultados
    for ent in doc.ents:
        if ent.label_ == "NUMERO_LEY":
            result["numero_ley"] = ent.text
        elif ent.label_ == "FECHA":
            # Normalizar la fecha a formato DD/MM/AAAA usando la función correspondiente
            fecha_norm = normalize_spacy_date(ent.text)
            result["fecha"] = fecha_norm if fecha_norm else ent.text
        elif ent.label_ == "EPIGRAFE":
            result["epigrafe"] = ent.text

    return result


## function to extract data

In [ ]:
def extract_metadata_from_txt(txt_path: str, nlp=None) -> dict:
    """
    Lee un archivo .txt, extrae metadatos y maneja errores.
    """
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            content = f.read()
    except FileNotFoundError:
        logging.error(f"Archivo no encontrado: {txt_path}")
        return None  # O un diccionario vacío, según prefieras
    except UnicodeDecodeError:
        logging.error(f"Error de codificación al leer: {txt_path}")
        return None
    except Exception as e:
        logging.error(f"Error inesperado al leer {txt_path}: {e}")
        return None

    if nlp:
        try:
            metadata = spacy_extract_metadata(content, nlp)
        except Exception as e:
            logging.error(f"Error al extraer metadatos con spaCy de {txt_path}: {e}")
            metadata = {}  # Diccionario vacío si falla spaCy
    else:
        metadata = {
            "numero_ley": None,
            "fecha": None,
            "tipo_ley": None,
            "epigrafe": None
        }

    return metadata

In [ ]:
# def extract_metadata_from_txt(txt_path: str, nlp=None) -> dict:
#     """
#     Lee un archivo .txt y extrae los metadatos (numero_ley, fecha, etc.)
#     usando tu pipeline híbrida o la que definas.
#     """
#     import os

#     # 1) Abrir y leer el archivo .txt
#     with open(txt_path, "r", encoding="utf-8") as f:
#         content = f.read()

#     # 2) Llamar a tu función de extracción (regex + spaCy)
#     #    Por ejemplo, si definiste extract_metadata_layout_hybrid:
#     if nlp:
#         metadata = spacy_extract_metadata(content, nlp)
#     else:
#         # Si no pasas un nlp, podrías usar solo regex
#         # metadata = extract_metadata_layout_regex(content)
#         metadata = {
#             "numero_ley": None,
#             "fecha": None,
#             "tipo_ley": None,
#             "epigrafe": None
#         }

#     return metadata


## USe

### Process the carpet

In [ ]:
def procesar_txt_en_carpeta(carpeta_txt: str, carpeta_salida: str, nlp=None):
    """
    Procesa archivos .txt, guarda metadatos en JSON y maneja errores.
    """
    os.makedirs(carpeta_salida, exist_ok=True)

    for archivo in os.listdir(carpeta_txt):
        if archivo.lower().endswith(".txt"):
            ruta_txt = os.path.join(carpeta_txt, archivo)
            metadatos = extract_metadata_from_txt(ruta_txt, nlp)

            if metadatos:  # Solo guarda si se extrajeron metadatos
                nombre_json = os.path.splitext(archivo)[0] + ".json"
                ruta_json = os.path.join(carpeta_salida, nombre_json)
                try:
                    with open(ruta_json, "w", encoding="utf-8") as f:
                        json.dump(metadatos, f, ensure_ascii=False, indent=4)
                    logging.info(f"Metadatos guardados en: {ruta_json}")
                except Exception as e:
                    logging.error(f"Error al guardar JSON para {archivo}: {e}")

In [ ]:
# def procesar_txt_en_carpeta(carpeta_txt: str, carpeta_salida: str, nlp=None):
#     """
#     Procesa todos los archivos .txt en la carpeta 'carpeta_txt',
#     extrae los metadatos y los guarda en archivos JSON en 'carpeta_salida'.
#     """
#     import os
#     import json

#     os.makedirs(carpeta_salida, exist_ok=True)  # Crea la carpeta de salida si no existe

#     for archivo in os.listdir(carpeta_txt):
#         if archivo.lower().endswith(".txt"):
#             ruta_txt = os.path.join(carpeta_txt, archivo)

#             # Llamar a la función que extrae metadatos de un .txt
#             metadatos = extract_metadata_from_txt(ruta_txt, nlp)

#             # Nombre del archivo JSON
#             nombre_json = os.path.splitext(archivo)[0] + ".json"
#             ruta_json = os.path.join(carpeta_salida, nombre_json)

#             # Guardar en JSON
#             with open(ruta_json, "w", encoding="utf-8") as f:
#                 json.dump(metadatos, f, ensure_ascii=False, indent=4)

#             print(f"Metadatos guardados en: {ruta_json}")

if __name__ == "__main__":
    # 1) Rutas
    carpeta_txt = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\leyes"
    carpeta_salida = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022"

    # 2) Cargar el modelo spaCy (si lo usas)
    import spacy
    nlp_trained = spacy.load("modelo_leyes_epigrafe")  # o la ruta a tu modelo

    # 3) Procesar la carpeta
    procesar_txt_en_carpeta(carpeta_txt, carpeta_salida, nlp_trained)

# Entrenar
nlp_trained = train_spacy_model(TRAIN_DATA, "modelo_leyes_epigrafe")

#Ejemplo de Uso
nlp_trained = load_spacy_model("modelo_leyes_epigrafe")

test_text = "2353 17ABR2024 LEY No. ... El Congreso de Colombia,"
test_text = fix_ocr_issues_for_training(test_text)  # Limpieza
doc = nlp_trained(test_text)
for ent in doc.ents:
    if ent.label_ == "FECHA":
        fecha_normalizada = normalize_spacy_date(ent.text)
        print("Fecha final:", fecha_normalizada)
    else:
        print(ent.text, ent.label_)